In [1]:
#importing libraries
import numpy as np
from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim
import torchbnn as bnn
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
#allocating datasets and model to GPU for speed's sake
is_available = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [3]:
class TripleOutputBNN(nn.Module):
    def __init__(self, no_of_neurones, dropout_prob):
        super(TripleOutputBNN, self).__init__()
        self.shared_layer = nn.Sequential( #this is the input layer
            bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=3, out_features=no_of_neurones),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
        )

        self.output_layer_n = nn.Sequential( #this is the output layer for n
            bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=no_of_neurones, out_features=no_of_neurones),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=no_of_neurones, out_features=1)
        )
        self.output_layer_f = nn.Sequential( #this is the output layer for f
            bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=no_of_neurones, out_features=no_of_neurones),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=no_of_neurones, out_features=1)
        )
        self.output_layer_r = nn.Sequential( #this is the output layer for r
            bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=no_of_neurones, out_features=no_of_neurones),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=no_of_neurones, out_features=1)
        )


    def forward(self, x): #this is the forward pass, run automatically when you call the model
        shared = self.shared_layer(x)
        n = self.output_layer_n(shared)
        f = self.output_layer_f(shared)
        r = self.output_layer_r(shared)
        return n , f , r


In [4]:
def test_model(model, inpt_test):
    """
    Test a Bayesian Neural Network model to produce predictions along with mean and standard deviation.

    Parameters:
    - model (torch.nn.Sequential): The trained neural network model.
    - func (callable): The target function to compare against the model's predictions.

    Returns:
    - mean_model_results (numpy.ndarray): Array of mean predictions from the model.
    - std_model_results (numpy.ndarray): Array of standard deviations of the predictions.
    - inpt_test (torch.Tensor): Testing dataset inputs.
    - outpt_test (torch.Tensor): Testing dataset targets.

    The testing data is moved to the device specified by the global `device` variable.
    """

    #producing predictions of model of testing data, as well as mean and standard deviation of predictions
    model.eval().cpu()
    n_pred, f_pred, r_pred = model(inpt_test)[0], model(inpt_test)[1], model(inpt_test)[2]

    n_pred, f_pred, r_pred = np.array([model(inpt_test)[0].detach().numpy() for _ in range(500)])[:,:,0].T, np.array([model(inpt_test)[1].detach().numpy() for _ in range(500)])[:,:,0].T, np.array([model(inpt_test)[2].detach().numpy() for _ in range(500)])[:,:,0].T

    mean_n_results, std_n_results = np.array([np.mean(n_pred[i]) for i in range(n_pred.shape[0])]), np.array([np.std(n_pred[i]) for i in range(n_pred.shape[0])])
    mean_f_results, std_f_results = np.array([np.mean(f_pred[i]) for i in range(f_pred.shape[0])]), np.array([np.std(f_pred[i]) for i in range(f_pred.shape[0])])
    mean_r_results, std_r_results = np.array([np.mean(r_pred[i]) for i in range(r_pred.shape[0])]), np.array([np.std(r_pred[i]) for i in range(r_pred.shape[0])])

    return mean_n_results, std_n_results, mean_f_results, std_f_results, mean_r_results, std_r_results


In [5]:
import random


def CreateData(filepath):

    #reading in data
    df = pd.read_csv(filepath)

    #creating input and output datasets
    inpt = df[['log_mstar', 'log_mdust_over_mstar','theta','wvl']]
    outpt = df[['n', 'flux', 'r','wvl']]

    #splitting data into training and testing datasets
    inpt_train, inpt_test, outpt_train, outpt_test = train_test_split(inpt, outpt, test_size=0.2, random_state=42)

    #resetting indices

    inpt_train.reset_index(drop=True, inplace=True)
    inpt_test.reset_index(drop=True, inplace=True)
    outpt_train.reset_index(drop=True, inplace=True)
    outpt_test.reset_index(drop=True, inplace=True)

    #sorting data by wavelength

    inpt_train = inpt_train.sort_values(by=['wvl']).drop(columns=['wvl'])
    inpt_test = inpt_test.sort_values(by=['wvl']).drop(columns=['wvl'])
    outpt_train = outpt_train.sort_values(by=['wvl']).drop(columns=['wvl'])
    outpt_test = outpt_test.sort_values(by=['wvl']).drop(columns=['wvl'])

    #converting data to tensors
    inpt_train, inpt_test, outpt_train, outpt_test = torch.Tensor(inpt_train.values), torch.Tensor(inpt_test.values), torch.Tensor(outpt_train.values), torch.Tensor(outpt_test.values)

    #moving data to GPU
    inpt_train, inpt_test, outpt_train, outpt_test = inpt_train.to(device), inpt_test.to(device), outpt_train.to(device), outpt_test.to(device)

    return inpt_train, inpt_test, outpt_train, outpt_test


In [6]:
model = torch.load(r"C:\Users\joshu\OneDrive\Desktop\Physics_Year_3\Final_year_project\Github\project_script\trained_models\radiative_transfer_model_mk1.pt")

model.eval()


TripleOutputBNN(
  (shared_layer): Sequential(
    (0): BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=3, out_features=1000, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
  )
  (output_layer_n): Sequential(
    (0): BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=1000, out_features=1000, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=1000, out_features=1, bias=True)
  )
  (output_layer_f): Sequential(
    (0): BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=1000, out_features=1000, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=1000, out_features=1, bias=True)
  )
  (output_layer_r): Sequential(
    (0): BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=1000, out_features=1000, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): BayesLinear(prior_mu=0, prior_sigma=0

In [7]:
filepath = r"C:\Users\joshu\OneDrive\Desktop\Physics_Year_3\Final_year_project\Github\project_script\data\radiative_transfer\data.csv"

inpt_train, inpt_test, outpt_train, outpt_test = CreateData(filepath)

print(inpt_train.shape, inpt_test.shape, outpt_train.shape, outpt_test.shape)

mean_n_results, std_n_results, mean_f_results, std_f_results, mean_r_results, std_r_results = test_model(model, inpt_test)

mse_loss = nn.MSELoss().to(device)
mean_n_mse, mean_f_mse, mean_r_mse = mse_loss(torch.Tensor(mean_n_results), outpt_test[:,0]), mse_loss(torch.Tensor(mean_f_results), outpt_test[:,1]), mse_loss(torch.Tensor(mean_r_results), outpt_test[:,2])

print(f"mean_n_mse: {mean_n_mse}, mean_f_mse: {mean_f_mse}", f"mean_r_mse: {mean_r_mse}")


torch.Size([10848, 3]) torch.Size([2712, 3]) torch.Size([10848, 3]) torch.Size([2712, 3])


mean_n_mse: 0.5117641091346741, mean_f_mse: 0.0006659212522208691 mean_r_mse: 12.143556594848633
